In [1]:
import time
start = time.time()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_union
from scipy.sparse import hstack
import time
import regex as re
import string
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import gc
from sklearn.metrics import roc_auc_score


In [2]:
train = pd.read_csv('/Users/dsg281/Desktop/test.csv')
train = train.drop_duplicates()

In [3]:
test = pd.read_csv("/Users/dsg281/Downloads/Dubai_Now_Sent/Comments_FROM_DB/Dubai NOW HM Comments 111232018.csv").dropna(subset=['Comments_EN'])


In [27]:
train.head()

,Questions,Target
0,l am deaf in senegal what do you the reason l ...,Complaint
1,kept receiving messages of deducted amounts wi...,Complaint
2,keeps hanging,Complaint
3,ive been using the app for min now and i still...,Complaint
4,hard to navigate around the app,Complaint


In [5]:
def cleaning(s):
    
    s = str(s)
    #s = str.split(s)
    s = s.lower()
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub(r'[^\w]', ' ', s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace(",","")
    s = s.replace("[\w*"," ")
    s = re.sub(r'https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE)
    s = re.sub(r'\<a href', ' ', s)
    s = re.sub(r'&amp;', '', s) 
    s = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', s)
    s = re.sub(r'[^\x00-\x7f]',r'',s) #removes arabic
    s = re.sub(r'<br />', ' ', s)
    s = re.sub(r'\'', ' ', s)
    
    return s

train['Questions'] = [cleaning(s) for s in train['Questions']]
test['Comments_EN'] = [cleaning(s) for s in test['Comments_EN']]

In [6]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words = 'english',
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=8000
)
# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=5000
)

In [7]:
train.columns

Index(['Questions', 'Target'], dtype='object')

In [8]:
word_vectorizer.fit(train['Questions'])
char_vectorizer.fit(train['Questions'])


TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.float32'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
# Train
train_word_features = word_vectorizer.transform(train['Questions'])
train_char_features = char_vectorizer.transform(train['Questions'])


In [10]:
# Sparse Matrix
train_features = hstack([
    train_char_features,
    train_word_features])

In [11]:
model_NB = MultinomialNB()

In [12]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(train_features, train.Target, train_size=0.75)

/Users/dsg281/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
model_NB = MultinomialNB()
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(train_features, train.Target, train_size=0.75)
model_NB.fit(X_train_tfidf, y_train_tfidf)
predictions_tfidf = model_NB.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, predictions_tfidf)
accuracy_tfidf

0.8316455696202532

In [14]:
#test = pd.read_csv("/Users/dsg281/Downloads/Dubai_Now_Sent/Comments_FROM_DB/Dubai NOW HM Comments 1242018.csv").dropna(subset=['Comments_EN'])


In [15]:
model_NB = MultinomialNB()

# Train the model, passing the x values, and the target (y)
# the vectorized variable contains all the test data.
model=model_NB.fit(train_features,train.Target)

In [16]:
#model1 = joblib.dump(model,"NB_Char_Word.pkl")


In [17]:
X_test1 = char_vectorizer.transform(test.Comments_EN)
X_test2 = word_vectorizer.transform(test.Comments_EN)

In [18]:
test_stack = hstack([X_test1,X_test2])

In [19]:
p =model_NB.predict(test_stack)

In [20]:
test['Predition'] = p

In [32]:
test.head()

,HB_BEAT_ID_BK,Date,Comments_EN,Predition
1,16376199,12/03/2018,its good one,Positive
2,16377296,12/03/2018,happy,Positive
3,16378152,12/03/2018,good aap thanks,Positive
4,16379351,12/03/2018,earlier format of dubai now was pretty userfri...,Positive
8,16381699,12/03/2018,the gui is not so modern it seems old,Complaint


In [24]:
print("Notebook took %0.2f minutes to Run"%((time.time() - start)/60))


Notebook took 4.76 minutes to Run


In [33]:
jupyter nbconvert --to html NOTEBOOK-NAME.ipynb

SyntaxError: invalid syntax (<ipython-input-33-0e87784aec10>, line 1)